## Stocks Download

In [1]:
stocks="XLE XLF XLU XLI XLK XLV XLY XLP XLB XOP XHB XME IYR IBB GLD"

In [2]:
import yfinance as yf
data = yf.download(stocks, start="1995-01-01", end="2021-09-01")

[*********************100%***********************]  15 of 15 completed


In [3]:
data.head()

Adj Close                                                          \
                 GLD IBB IYR XHB        XLB        XLE        XLF        XLI   
Date                                                                           
1998-12-22       NaN NaN NaN NaN  12.525131  13.500203  12.079448  15.476096   
1998-12-23       NaN NaN NaN NaN  12.656684  13.781269  12.257560  15.746142   
1998-12-24       NaN NaN NaN NaN  12.947957  13.708741  12.338526  15.953880   
1998-12-28       NaN NaN NaN NaN  12.835207  13.636205  12.176604  16.036970   
1998-12-29       NaN NaN NaN NaN  13.070112  13.772202  12.306145  16.265474   

                                  ... Volume                                \
                  XLK        XLP  ...    XLE     XLF   XLI     XLK     XLP   
Date                              ...                                        
1998-12-22  24.669796  15.752134  ...  15200   55887   600  300500  150300   
1998-12-23  25.259169  16.132940  ...  67800   78784   900  338300   44000   
1998-12-24  25.162939  16.105070  ...  12300   43824  1400  243700   45500   
1998-12-28  25.235113  16.012196  ...  13500   51948  7400  309300   25400   
1998-12-29  25.307283  16.365131  ...  22000  100819  1400  123800  123500   

                                         
              XLU    XLV    XLY XME XOP  
Date                                     
1998-12-22   7900   5700   3700 NaN NaN  
1998-12-23  24100  18100  12000 NaN NaN  
1998-12-24  23800   4900   1600 NaN NaN  
1998-12-28   3100  15500   8600 NaN NaN  
1998-12-29  12000   5300   1100 NaN NaN  

[5 rows x 90 columns]

In [4]:
adj_close=data['Adj Close']

### Set up dataframe and train/validate/test splits
We reserve data afer 2020 as our test dataset, and data from 2016/10/1 to 2019/12/31 for validation and rest as train dataset

In [6]:
etfs=adj_close[['XLB','XLE','XLF','XLI','XLK','XLP','XLU','XLV','XLY']]

etfs.loc[:, 'split']='val'
etfs.loc[etfs.index > '2020-01-01', 'split']='test'
etfs.loc[etfs.index < '2016-10-01', 'split']='train'

In [9]:
import numpy as np
etfs_log_chg = np.log1p(etfs[etfs.columns[:-2]].pct_change())
etfs_log_chg['split'] =  etfs.split
etfs_log_chg = etfs_log_chg.iloc[1:,:]

In [176]:
# save csv files
etfs.to_csv('sector_etf.csv')
etfs_log_chg.to_csv('sector_etf_logchg.csv')